In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import MatrixBasedConvolution as mbc

## Convolution parameters

In [2]:
inputs_shape_1d=(1,20)
inputs_shape_2d=(1,5,5,1)
ARGS={"kernel_size":3,
"stride":1,
"padding":'same',
"use_phi":True,
"activation":None,
"use_lambda_out":False,
"use_lambda_in":False}

## Convalution 1D and 2D outputs

In [3]:
print("1D Convolution building...")
con1d=mbc.matrix_conv_1d(**ARGS)
con1d.build(inputs_shape_1d)
print("2D Convolution building...")
con2d=mbc.matrix_conv_2d(**ARGS)
con2d.build(inputs_shape_2d)

1D Convolution building...
2D Convolution building...


In [4]:
print("inputs 1D generation...")
inputs_1d=tf.random.uniform(shape=inputs_shape_1d,minval=0,maxval=1)
print("inputs 2D generation...")
inputs_2d=tf.random.uniform(shape=inputs_shape_2d,minval=0,maxval=1)


inputs 1D generation...
inputs 2D generation...


In [5]:
ouputs_mbc_1d=con1d.conv_jit(inputs_1d).numpy()
print(ouputs_mbc_1d.shape)
ouputs_mbc_2d=con2d.conv_jit(inputs_2d).numpy()[0,:,:,0]
print(ouputs_mbc_2d.shape)

(1, 20)
(5, 5)


## Comparison between standard conv1d/conv2d and matrix based conv1d/conv2d

In [6]:
kernel=tf.reshape(con2d.kernel,shape=(ARGS.get('kernel_size'),ARGS.get('kernel_size')))
kernel.shape

TensorShape([3, 3])

In [7]:
ouputs_classical=mbc.classical_convolution(img=inputs_2d[0,:,:,0],kernel=kernel.numpy())
ouputs_classical.shape


(5, 5)

In [8]:
ouputs_mbc_2d

array([[ 0.12753487, -1.1348276 , -0.53910995, -0.58248913, -1.1491637 ],
       [-0.74660265, -0.8602542 , -1.134738  , -1.028091  , -1.1955901 ],
       [-0.26114795, -0.78310037, -1.1808732 , -0.82915115, -1.1672337 ],
       [-0.468871  , -0.851454  , -0.2947304 , -1.4059453 , -0.48479998],
       [-0.11618054, -0.4731935 , -0.62337714, -0.23381549, -0.6115488 ]],
      dtype=float32)

In [9]:
ouputs_classical

array([[ 0.12753487, -1.1348275 , -0.5391099 , -0.58248913, -1.1491637 ],
       [-0.74660265, -0.86025417, -1.134738  , -1.028091  , -1.1955903 ],
       [-0.26114798, -0.78310037, -1.1808732 , -0.82915103, -1.1672338 ],
       [-0.46887097, -0.851454  , -0.29473042, -1.4059454 , -0.48479998],
       [-0.11618054, -0.4731935 , -0.6233772 , -0.23381549, -0.6115488 ]],
      dtype=float32)

In [10]:
if np.allclose(ouputs_classical,ouputs_mbc_2d,atol=1e-16):
    print("The outputs are the same")
    

The outputs are the same
